### まずは有効なすべてのTickerとTimestampの組合せを取得する

In [ ]:
WINDOW = 200
RADIUS = 1
BATCH_SIZE = 128

In [ ]:
import numpy as np
import pandas as pd

def generate_data(df):
    target_indices = []
    timestamps = []
    
    for i in range(WINDOW, len(df)):
        target_indices.append(i)
        timestamps.append(df.iloc[i].name)
        
    return target_indices, timestamps

In [ ]:
import os
import joblib
from tqdm import tqdm
from joblib import Parallel, delayed

input_dir = 'history'

def process_file(filename, input_dir):
    """各ファイルを処理する関数"""
    if not filename.endswith('.joblib'):
        return None
    
    file_path = os.path.join(input_dir, filename)
    history_data = joblib.load(file_path)
    
    ticker = history_data['ticker']
    history_df = history_data['history_df']
    
    # データの作成
    target_indices, timestamps = generate_data(history_df)
    
    return {
        'tickers': [ticker] * len(target_indices),
        'target_indices': target_indices,
        'timestamps': timestamps
    }

# 並列処理の実行
files = [f for f in os.listdir(input_dir) if f.endswith('.joblib')]

# n_jobs=-1で全CPUコアを使用
results = Parallel(n_jobs=-1)(
    delayed(process_file)(filename, input_dir) 
    for filename in tqdm(files, desc="Processing files")
)

# 結果の結合
tickers = []
target_indices = []
timestamps = []

for result in results:
    if result is not None:
        tickers.extend(result['tickers'])
        target_indices.extend(result['target_indices'])
        timestamps.extend(result['timestamps'])

### test_start_timestamp よりも前のデータを学習に使用する

In [ ]:
from datetime import datetime, timedelta
test_start_timestamp = pd.Timestamp(datetime.now() - timedelta(days=365)).tz_localize('Asia/Tokyo')

is_train = [timestamp < test_start_timestamp for timestamp in timestamps]
train_indices = np.arange(len(tickers))[is_train]

from sklearn.model_selection import train_test_split
train_indices, val_indices = train_test_split(train_indices)

is_test = [timestamp >= test_start_timestamp for timestamp in timestamps]
test_indices = np.arange(len(tickers))[is_test]

### データセットの作成

In [ ]:
import numpy as np

def create_image(df):
    size = len(df)
    array = np.zeros((size, size, 3), dtype=np.uint8)

    epsilon = 1e-10
    log_high = np.log(df['High'].values + epsilon)
    log_low = np.log(df['Low'].values + epsilon)
    log_open = np.log(df['Open'].values + epsilon)
    log_close = np.log(df['Close'].values + epsilon)
    
    # NaNを含む時刻のマスクを作成
    nan_mask = (np.isnan(log_high) | np.isnan(log_low) | 
                np.isnan(log_open) | np.isnan(log_close))
    
    global_min = np.nanmin(log_low)
    global_max = np.nanmax(log_high)
    center = (global_min + global_max) / 2

    price_min = center - RADIUS
    price_max = center + RADIUS

    # 価格を画像のy座標にマッピング（0が上端で高値、size-1が下端で安値）
    def price_to_y(price):
        # price_maxが上端(0)、price_minが下端(size-1)
        y = (price_max - price) / (price_max - price_min) * (size - 1)
        # NaNの場合は-1を返す（後で無効化するため）
        y = np.where(np.isnan(price), -1, y)
        return np.clip(y, -1, size - 1).astype(int)
    
    # 各時点の価格をy座標に変換
    y_high = price_to_y(log_high)
    y_low = price_to_y(log_low)
    y_open = price_to_y(log_open)
    y_close = price_to_y(log_close)
    
    # x座標（時間軸）のインデックス配列
    x_indices = np.arange(size)
    
    # チャンネル1: HighからLowまでの範囲を255で塗りつぶす
    for i in range(size):
        if not nan_mask[i]:
            # y_high[i]からy_low[i]まで塗りつぶす（y_high <= y_low なので注意）
            array[y_high[i]:y_low[i]+1, i, 0] = 255
    
    # チャンネル2: Open < Closeの場合、OpenからCloseまでを255で塗りつぶす（陽線）
    bullish = log_open < log_close  # 上昇（陽線）
    for i in range(size):
        if bullish[i] and not nan_mask[i]:
            # CloseがOpenより高い（yは小さい）
            y_min = min(y_close[i], y_open[i])  # 上端
            y_max = max(y_close[i], y_open[i])  # 下端
            array[y_min:y_max+1, i, 1] = 255
    
    # チャンネル3: Close < Openの場合、CloseからOpenまでを255で塗りつぶす（陰線）
    bearish = log_close < log_open  # 下降（陰線）
    for i in range(size):
        if bearish[i] and not nan_mask[i]:
            # OpenがCloseより高い（yは小さい）
            y_min = min(y_open[i], y_close[i])  # 上端
            y_max = max(y_open[i], y_close[i])  # 下端
            array[y_min:y_max+1, i, 2] = 255
    
    return array

In [ ]:
class Dataset:
    def __init__(self, tickers, target_indices, timestamps, valid_indices, hist_dir='history'):
        self.tickers = tickers
        self.target_indices = target_indices
        self.timestamps = timestamps
        self.hist_dir = hist_dir
        self.valid_indices = valid_indices
        self.window = WINDOW

    def __len__(self):
        return len(self.valid_indices)

    def __getitem__(self, idx):
        valid_idx = self.valid_indices[idx]
        
        ticker = self.tickers[valid_idx]
        timestamp = self.timestamps[valid_idx].isoformat()
        target_indice = self.target_indices[valid_idx]
        
        history_df = joblib.load(os.path.join(self.hist_dir, f"{ticker}.joblib"))['history_df']
        img = create_image(history_df.iloc[target_indice - self.window: target_indice])
        target_row = history_df.iloc[target_indice]
        target_change = np.log(target_row['Close'] + 1e-10) - np.log(target_row['Open'] + 1e-10)
        # nanの場合は0に置換
        if np.isnan(target_change):
            target_change = 0.0
        return img, target_change, ticker, timestamp

In [ ]:
train_dataset = Dataset(tickers, target_indices, timestamps, train_indices)
val_dataset = Dataset(tickers, target_indices, timestamps, val_indices)
test_dataset = Dataset(tickers, target_indices, timestamps, test_indices)

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=8)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8)

### モデル

In [ ]:
from modeling import SimpleCNNRegressor
import torch
from torch import nn, optim

model = SimpleCNNRegressor()

device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5)

### 学習ループ

In [ ]:
import torch
import copy
from tqdm import tqdm
import warnings

# NaNチェック用のヘルパー関数
def check_nan(tensor, name="tensor"):
    """テンソルにNaNが含まれているかチェック"""
    if torch.isnan(tensor).any():
        return True
    return False

def check_inf(tensor, name="tensor"):
    """テンソルにInfが含まれているかチェック"""
    if torch.isinf(tensor).any():
        return True
    return False

# 学習ループ
def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    
    for batch_idx, (images, targets, _, _) in enumerate(tqdm(dataloader, desc='train')):
        images = images.to(device).permute(0, 3, 1, 2).float() / 255.0
        targets = targets.to(device).float().unsqueeze(1)
        
        # 入力データのNaNチェック
        if check_nan(images, "images") or check_inf(images, "images"):
            warnings.warn(f"NaN or Inf detected in images at batch {batch_idx}")
            raise ValueError(f"NaN or Inf detected in input images at batch {batch_idx}")
        
        if check_nan(targets, "targets") or check_inf(targets, "targets"):
            warnings.warn(f"NaN or Inf detected in targets at batch {batch_idx}")
            raise ValueError(f"NaN or Inf detected in targets at batch {batch_idx}")
        
        optimizer.zero_grad()
        outputs = model(images)
        
        # モデル出力のNaNチェック
        if check_nan(outputs, "outputs") or check_inf(outputs, "outputs"):
            warnings.warn(f"NaN or Inf detected in model outputs at batch {batch_idx}")
            raise ValueError(f"NaN or Inf detected in model outputs at batch {batch_idx}")
        
        loss = criterion(outputs, targets)
        
        # 損失のNaNチェック
        if check_nan(loss, "loss") or check_inf(loss, "loss"):
            warnings.warn(f"NaN or Inf detected in loss at batch {batch_idx}")
            raise ValueError(f"NaN or Inf detected in loss at batch {batch_idx}")
        
        loss.backward()
        
        # 勾配のNaNチェック（オプション）
        for name, param in model.named_parameters():
            if param.grad is not None:
                if check_nan(param.grad, f"gradient of {name}") or check_inf(param.grad, f"gradient of {name}"):
                    warnings.warn(f"NaN or Inf detected in gradients of {name} at batch {batch_idx}")
                    raise ValueError(f"NaN or Inf detected in gradients of {name} at batch {batch_idx}")
        
        optimizer.step()
        
        total_loss += loss.item()
        
    return total_loss / len(dataloader)

# 評価
def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    
    with torch.no_grad():
        for batch_idx, (images, targets, _, _) in enumerate(tqdm(dataloader, desc='eval')):
            images = images.to(device).permute(0, 3, 1, 2).float() / 255.0
            targets = targets.to(device).float().unsqueeze(1)
            
            # 入力データのNaNチェック
            if check_nan(images, "images") or check_inf(images, "images"):
                warnings.warn(f"NaN or Inf detected in images at batch {batch_idx}")
                raise ValueError(f"NaN or Inf detected in input images at batch {batch_idx}")
            
            if check_nan(targets, "targets") or check_inf(targets, "targets"):
                warnings.warn(f"NaN or Inf detected in targets at batch {batch_idx}")
                raise ValueError(f"NaN or Inf detected in targets at batch {batch_idx}")
            
            outputs = model(images)
            
            # モデル出力のNaNチェック
            if check_nan(outputs, "outputs") or check_inf(outputs, "outputs"):
                warnings.warn(f"NaN or Inf detected in model outputs at batch {batch_idx}")
                raise ValueError(f"NaN or Inf detected in model outputs at batch {batch_idx}")
            
            loss = criterion(outputs, targets)
            
            # 損失のNaNチェック
            if check_nan(loss, "loss") or check_inf(loss, "loss"):
                warnings.warn(f"NaN or Inf detected in loss at batch {batch_idx}")
                raise ValueError(f"NaN or Inf detected in loss at batch {batch_idx}")
            
            total_loss += loss.item()
            
    return total_loss / len(dataloader)

# 早期停止の設定
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0):
        """
        Args:
            patience: 改善が見られない場合に待つエポック数
            min_delta: 改善とみなす最小の変化量
        """
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        
    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0
            
        return self.early_stop

# 学習実行
num_epochs = 100
early_stopping = EarlyStopping(patience=10, min_delta=1e-4)
best_model_state = None
best_val_loss = float('inf')

try:
    for epoch in range(num_epochs):
        train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
        val_loss = evaluate(model, val_loader, criterion, device)
        
        # 損失値のNaNチェック
        if torch.isnan(torch.tensor(train_loss)) or torch.isinf(torch.tensor(train_loss)):
            raise ValueError(f"NaN or Inf detected in training loss at epoch {epoch+1}")
        
        if torch.isnan(torch.tensor(val_loss)) or torch.isinf(torch.tensor(val_loss)):
            raise ValueError(f"NaN or Inf detected in validation loss at epoch {epoch+1}")
        
        scheduler.step(val_loss)
        
        print(f'Epoch {epoch+1}/{num_epochs}:')
        print(f'  Train Loss: {train_loss:.6f}')
        print(f'  Val Loss: {val_loss:.6f}')
        
        # ベストモデルの保存
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = copy.deepcopy(model.state_dict())
            print(f'  → Best model saved! (val_loss: {val_loss:.6f})')
        
        # 早期停止のチェック
        if early_stopping(val_loss):
            print(f'\nEarly stopping triggered at epoch {epoch+1}')
            break

except ValueError as e:
    print(f"\n{'='*50}")
    print(f"Training interrupted due to NaN/Inf detection:")
    print(f"  {str(e)}")
    print(f"{'='*50}\n")
    
    # デバッグ情報の出力
    print("Debug information:")
    print(f"  Current epoch: {epoch+1}")
    print(f"  Current learning rate: {optimizer.param_groups[0]['lr']}")
    
    # モデルのパラメータの状態をチェック
    print("\nModel parameter statistics:")
    for name, param in model.named_parameters():
        if param is not None:
            print(f"  {name}:")
            print(f"    Shape: {param.shape}")
            print(f"    Mean: {param.mean().item():.6f}")
            print(f"    Std: {param.std().item():.6f}")
            print(f"    Min: {param.min().item():.6f}")
            print(f"    Max: {param.max().item():.6f}")
            print(f"    Has NaN: {torch.isnan(param).any().item()}")
            print(f"    Has Inf: {torch.isinf(param).any().item()}")
    
    raise

# 最良のモデルを復元
if best_model_state is not None:
    model.load_state_dict(best_model_state)
    print(f'Best model restored (val_loss: {best_val_loss:.6f})')
